In [60]:
import LCCall as LCC
import LCAnalyses as LCA
import LCPlot as LCP
import importlib
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import patches
importlib.reload(LCC)
importlib.reload(LCA)
importlib.reload(LCP)
%matplotlib tk

In [57]:
k = "EC53"
mod_date = "191021"
region = "SERPM"
input_dir = '/mnt/Secdrive/TSmain/SERPM/EC53/EC53/'
output_dir = '/mnt/Secdrive/TSmain/SERPM/Results_'+mod_date+'/EC53/'
dates_K, mags_K, noises_K = LCC.EC53IR("UKIRT_K",imeunit="day",addLiverpool=0,timeunit="day")
dates_H, mags_H, noises_H = LCC.EC53IR("UKIRT_H",imeunit="day",addLiverpool=1,timeunit="day")
dates_J, mags_J, noises_J = LCC.EC53IR("UKIRT_J",imeunit="day",addLiverpool=0,timeunit="day")
dates_LH, mags_LH, noises_LH = LCC.EC53IR("Liverpool_H",imeunit="day",timeunit="day")

JDs,Js,Std_Js,Hs,Std_Hs,a,Ks,Std_Ks,b,J_Hs,J_H_errs,H_Ks,H_K_errs, = np.loadtxt(input_dir +"/EC53_UKIRT_JHK_combined.dat", unpack=True, dtype='float')
dates_850, fluxes_850, noises_850 = LCC.JCMTTRansientSource(k,"SERPM","850",mod_date)

In [1]:
Obs_terms = [[0,3],[4,23],[24,50],[51,80],[81,107],[108,111],[112,149],[150, 151],[152,214],[215,257],[258,283],[284,316]]
Obs_terms = [[0,1],[2,11],[12,25],[26,40],[41,54],[55,56],[57,75],[76,76][77,104],[105,125],[126,138],[139,155]]
#Obs_terms = [[0,3],[4,50],[51,80],[81,107],[108,111],[112,149],[150, 151],[152,214],[215,257],[258,283],[284,316]]

TypeError: list indices must be integers or slices, not tuple

In [41]:
def makeArrow(ax,pos1,pos2,direction):
    delta = 0.0001 if direction >= 0 else -0.0001
    posx = (pos1[0]+pos2[0])/2
    posy = (pos1[1]+pos2[1])/2
    ax.arrow(posx,posy,delta,delta*(pos1[1]-pos2[1])/(pos1[0]-pos2[0]),
             head_width=0.01,head_length=0.01,color='black',alpha=0.5)

In [ ]:
print(np.mean(J_H_errs),np.mean(H_K_errs))

In [54]:
TCs_terms = []
TCs_av_terms = []
LCs_terms = []
mags_J_terms= [];mags_H_terms= [];mags_K_terms= []
i = 0
plt.close()
fig, ax = plt.subplots(figsize=[9,7])
colormap = plt.cm.Paired(np.linspace(0,1,12))
for Obs_term in Obs_terms:
    TCs_terms.append([H_Ks[Obs_term[0]:(Obs_term[1]+1)],J_Hs[Obs_term[0]:(Obs_term[1]+1)],
                          J_H_errs[Obs_term[0]:(Obs_term[1]+1)],H_K_errs[Obs_term[0]:(Obs_term[1]+1)]])
    TCs_av_terms.append([np.mean(H_Ks[Obs_term[0]:(Obs_term[1]+1)]),np.mean(J_Hs[Obs_term[0]:(Obs_term[1]+1)]),
                          2*np.std(H_Ks[Obs_term[0]:(Obs_term[1]+1)]),2*np.std(J_Hs[Obs_term[0]:(Obs_term[1]+1)])])
    mags_J_terms.append([Js[Obs_term[0]:(Obs_term[1]+1)],Std_Js[Obs_term[0]:(Obs_term[1]+1)]])
    mags_H_terms.append([Hs[Obs_term[0]:(Obs_term[1]+1)],Std_Hs[Obs_term[0]:(Obs_term[1]+1)]])
    mags_K_terms.append([Ks[Obs_term[0]:(Obs_term[1]+1)],Std_Ks[Obs_term[0]:(Obs_term[1]+1)]])
    LCs_terms.append([Ks[Obs_term[0]:(Obs_term[1]+1)],Std_Ks[Obs_term[0]:(Obs_term[1]+1)]])
beg = 0
end = len(TCs_terms)
for j in range(beg,end):
    i = j +1
    #if j == beg:
    #    ax.plot([x[0] for x in TCs_av_terms[beg:end]],[x[1] for x in TCs_av_terms[beg:end]],
    #            alpha=0.5,color='black', linewidth=2)
    ax.scatter(TCs_terms[j][0],TCs_terms[j][1],color=colormap[j],alpha=0.2)
    #ells = patches.Ellipse((TCs_av_terms[j][0],TCs_av_terms[j][1]),TCs_av_terms[j][2],TCs_av_terms[j][3],
    #                         alpha=0.3,color=colormap[j])
    ax.errorbar(TCs_av_terms[j][0],TCs_av_terms[j][1],xerr=TCs_av_terms[j][2],yerr=TCs_av_terms[j][3],
                fmt="o-",label=str(i),color=colormap[j],alpha=1)
    #if not j == beg:
    #    makeArrow(ax, TCs_av_terms[j-1],TCs_av_terms[j],TCs_av_terms[j][0]-TCs_av_terms[j-1][0])
    #ax.add_artist(ells)
ell_err = patches.Ellipse((2.2,2.1),2*np.mean(H_K_errs),2*np.mean(J_H_errs),
                             alpha=0.3,color="black")
ax.add_artist(ell_err)
x = np.linspace(1.7,0.01,2.4)
plt.plot(H_Ks,H_Ks*0.58+1.08,dashes=[5,5],linewidth=2,color='grey')
plt.arrow(1.9,2.5,0.065,0.11,head_width=0.01,head_length=0.005)
plt.annotate(r"1 A$_{V}$",xy=[1.9,2.57], fontsize=13)
#arrowsz =[TCs_av_terms[3][0]-TCs_av_terms[2][0]+0.02, TCs_av_terms[3][1]-TCs_av_terms[2][1]+0.02]
#plt.arrow(TCs_av_terms[2][0],TCs_av_terms[2][1],arrowsz[0],arrowsz[1],width=0.01,head_width=0.02,head_length=0.03,alpha=0.5,color="blue")
#arrowsz =[TCs_av_terms[1][0]-TCs_av_terms[0][0]-0.02, TCs_av_terms[1][1]-TCs_av_terms[0][1]-0.02]
#plt.arrow(TCs_av_terms[0][0],TCs_av_terms[0][1],arrowsz[0],arrowsz[1],width=0.01,head_width=0.02,head_length=0.03,alpha=0.5,color="red")
#arrowsz =[TCs_av_terms[4][0]-TCs_av_terms[3][0]-0.02, TCs_av_terms[4][1]-TCs_av_terms[3][1]-0.02]
#plt.arrow(TCs_av_terms[3][0],TCs_av_terms[3][1],arrowsz[0],arrowsz[1],width=0.01,head_width=0.02,head_length=0.03,alpha=0.5,color="red")
#arrowsz =[TCs_av_terms[7][0]-TCs_av_terms[6][0]-0.02, TCs_av_terms[7][1]-TCs_av_terms[6][1]-0.02]
#plt.arrow(TCs_av_terms[6][0],TCs_av_terms[6][1],arrowsz[0],arrowsz[1],width=0.01,head_width=0.02,head_length=0.03,alpha=0.5,color="red")
#arrowsz =[TCs_av_terms[11][0]-TCs_av_terms[10][0]-0.02, TCs_av_terms[11][1]-TCs_av_terms[10][1]-0.02]
#plt.arrow(TCs_av_terms[10][0],TCs_av_terms[10][1],arrowsz[0],arrowsz[1],width=0.01,head_width=0.02,head_length=0.03,alpha=0.5,color="red")
#arrowsz =[TCs_av_terms[6][0]-TCs_av_terms[5][0]+0.02, TCs_av_terms[6][1]-TCs_av_terms[5][1]+0.02]
#plt.arrow(TCs_av_terms[5][0],TCs_av_terms[5][1],arrowsz[0],arrowsz[1],width=0.01,head_width=0.02,head_length=0.03,alpha=0.5,color="blue")
#arrowsz =[TCs_av_terms[9][0]-TCs_av_terms[8][0]+0.02, TCs_av_terms[9][1]-TCs_av_terms[8][1]+0.02]
#plt.arrow(TCs_av_terms[8][0],TCs_av_terms[8][1],arrowsz[0],arrowsz[1],width=0.01,head_width=0.02,head_length=0.03,alpha=0.5,color="blue")
#plt.arrow(TCs_av_terms[2][0],TCs_av_terms[2][1],-0.16,-0.13,head_width=0.01,head_length=0.005,alpha=0.7)

ax.legend(loc="upper left",fontsize=13)
ax.set_ylim([2.02,2.68])
ax.set_xlim([1.71,2.31])
ax.set_ylabel("J-H",size=17)
ax.set_xlabel("H-K",size=17)
plt.tick_params(axis='x',labelsize=17)
plt.tick_params(axis='y',labelsize=17)
fig.savefig(output_dir + region+'_'+k+'_TCD_191216_Scatter.pdf')

/home/lyhee/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:36: DeprecationWarning: object of type <class 'float'> cannot be safely interpreted as an integer.


In [ ]:
plt.close(fig)
fig, ax = plt.subplots(figsize=[9,7])

for j in range(0,len(TCs_terms)):
    i = j +1
    ax.scatter(TCs_terms[j][0],mags_H_terms[j][0],color=colormap[j],alpha=0.2)
    #ells = patches.Ellipse((TCs_av_terms[j][0],TCs_av_terms[j][1]),TCs_av_terms[j][2],TCs_av_terms[j][3],
    #                         alpha=0.3,color=colormap[j])
    ax.errorbar(TCs_av_terms[j][0],np.mean(mags_H_terms[j][0]),
                yerr=np.std(mags_H_terms[j][0]),xerr=TCs_av_terms[j][2],
                fmt="o-",label=str(i),color=colormap[j],alpha=1)
ell_err = patches.Ellipse((1.8,14.2),2*np.mean(H_K_errs),2*np.mean(Std_Ks),
                             alpha=0.3,color="black")
ax.add_artist(ell_err)
ax.legend(loc="upper right",fontsize=13)
ax.set_ylabel("K",size=17)
ax.set_xlabel("H-K",size=17)
plt.gca().invert_yaxis()
plt.tick_params(axis='x',labelsize=17)
plt.tick_params(axis='y',labelsize=17)
fig.savefig(output_dir + region+'_'+k+'_KtoK-H_191204_Scatter.pdf')

In [96]:
importlib.reload(LCP)
region = "SERPM"
wl = "K-band"
plt.close()
fig, ax1 = plt.subplots()

colorH = "green"
colorHK = "mediumblue"
ax1.set_xlabel("MJD")
ax1.set_ylabel("Liverpool H-band [mag]",color=colorH)
ax1.errorbar([x-2400000.5 for x in dates_LH], mags_LH,yerr=noises_LH,
             color=colorH, fmt='o')
ax1.tick_params(axis='y',labelcolor=colorH)
plt.gca().invert_yaxis()

ax2 = ax1.twinx()

ax2.set_ylabel("(H-K)",color=colorHK)
ini = 150
ax2.errorbar([x-2400000.5 for x in JDs[ini:]], H_Ks[ini:], yerr=H_K_errs[ini:],
             color=colorHK, fmt='o',fillstyle='none')
ax2.tick_params(axis='y',labelcolor=colorHK)
#plt.xlim([58200,58800])
fig.tight_layout()
plt.show()


plt.savefig(output_dir + region+'_'+k+'_K_H-K_LC_191226.pdf')

317

In [84]:
output_dir + region+'_'+k+'_TCD_191029_arrow.pdf'

'/mnt/Secdrive/TSmain/SERPM/Results_191021/EC53/SERPM_EC53_TCD_191029_arrow.pdf'

In [52]:
importlib.reload(LCP)
region = "SERPM"
wl = "K-band"
plt.close()
plt.figure(figsize=[9,7])
for j in range(0,len(TCs_terms)):
    i = j +1
    LCP.LCplot([x-2400000.5 for x in JDs[Obs_terms[j][0]:(Obs_terms[j][1]+1)]], LCs_terms[j][0],LCs_terms[j][1]
               ,mod_date,region,k,wl,mag=1,save=0,newplot=0,markersize=5,capsize=0
               ,color=colormap[j],ecolor = 'black',figureindex=1,alpha = 1,label=str(i))
    
plt.legend(loc="upper left",fontsize=13)
plt.gca().invert_yaxis()
plt.ylabel("Magnitude [mag]",size=17)
plt.xlabel("MJD",size=17)

plt.tick_params(axis='x',labelsize=17)
plt.tick_params(axis='y',labelsize=17)
plt.savefig(output_dir + region+'_'+k+'_K_LC_191216.pdf')

In [ ]:
importlib.reload(LCP)
region = "SERPM"
wl = "K-band"
plt.close()
plt.figure(figsize=[9,7])
for j in range(0,len(TCs_terms)):
    i = j +1
    correction = [(x-JDs[0]) for x in JDs[Obs_terms[j][0]:(Obs_terms[j][1]+1)]][0]/365.24
    LCP.LCplot([(x-JDs[0])%543/543 for x in JDs[Obs_terms[j][0]:(Obs_terms[j][1]+1)]]
               ,LCs_terms[j][0],LCs_terms[j][1]
               ,mod_date,region,k,wl,mag=1,save=0,newplot=0,markersize=5,capsize=0
               ,color=colormap[j],ecolor = 'black',figureindex=1,alpha = 1,label=str(i))
plt.legend(loc="upper left",fontsize=13)
plt.gca().invert_yaxis()
plt.xlabel("Phase")
plt.tick_params(axis='x',labelsize=17)
plt.tick_params(axis='y',labelsize=17)
plt.ylabel("Magnitude [mag]",size=17)
plt.xlabel("Phase",size=17)
plt.savefig(output_dir + region+'_'+k+'_K_543PD_191014.pdf')

In [ ]:
#Liverpool H & H-K color variation time series
importlib.reload(LCP)
region = "SERPM"
wl = "LiverpoolH"
plt.close()
plt.figure(figsize=[9,7])


LCP.LCplot([x-dates_LH[0] for x in dates_LH], mags_LH,LCs_terms[j][1]
               ,mod_date,region,k,wl,mag=1,save=0,newplot=0,markersize=5,capsize=0
               ,color=colormap[j],ecolor = 'black',figureindex=1,alpha = 1,label=str(i))
    
plt.legend(loc="upper left",fontsize=13)
plt.gca().invert_yaxis()
plt.ylabel("Magnitude [mag]",size=17)
plt.xlabel("MJD",size=17)

plt.tick_params(axis='x',labelsize=17)
plt.tick_params(axis='y',labelsize=17)
plt.savefig(output_dir + region+'_'+k+'_'+wl+'_LC_191216.pdf')